In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [18]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D20.parquet')
test_day.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.260847,75,ABEV3.SA,-0.029034,0.509553,1.0,0.485255,1.0,0.639548,0.545296,0.029034,0.000000,43.244009,0.468107,0.527474,69.813010,-30.186990,-0.000988,-0.000683,-0.010422,15246
1,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.265805,455,ABEV3.SA,0.009223,0.508799,1.0,0.494478,1.0,0.657994,0.550931,0.000000,0.009223,47.299662,0.468107,0.527474,68.543373,-31.456627,-0.001096,-0.000769,-0.013089,14791
2,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.262261,5005,ABEV3.SA,-0.006593,0.507753,1.0,0.487885,1.0,0.652734,0.556021,0.006593,0.000000,44.667003,0.468107,0.527474,66.781944,-33.218056,-0.001227,-0.000863,-0.015941,9786
3,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.255173,3033,ABEV3.SA,-0.013185,0.506101,1.0,0.474700,1.0,0.652734,0.560856,0.013185,0.000000,39.583155,0.468107,0.527474,63.998121,-36.001879,-0.001415,-0.000976,-0.017728,6753
4,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.251629,11602,ABEV3.SA,-0.006593,0.504201,1.0,0.468107,1.0,0.606585,0.563143,0.006593,0.000000,37.143972,0.468107,0.527474,60.798215,-39.201785,-0.001661,-0.001116,-0.016919,-4849


In [19]:
test_day['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [20]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.tail(15)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5682,2024-11-01,12.51,12.65,12.48,12.54,11.884937,29027800,ABEV3.SA,-0.100000,12.862507,NaN,12.54,1.0,13.85,12.761406,0.100000,0.00,25.201017,12.48,13.19,53.874293,-46.125707,0.085035,0.109780,-0.002567,-2368936575
5683,2024-11-04,12.59,12.71,12.51,12.67,12.008146,19442900,ABEV3.SA,0.130000,12.852882,NaN,12.67,1.0,13.93,12.819836,0.000000,0.13,37.747293,12.48,13.19,52.518610,-47.481390,0.077261,0.103276,-0.003254,-2388379475
5684,2024-11-05,12.60,12.74,12.57,12.68,12.017624,18352000,ABEV3.SA,0.010000,12.844238,NaN,12.68,1.0,14.42,12.899844,0.000000,0.01,38.660509,12.48,13.09,59.711101,-40.288899,0.069600,0.096541,-0.003673,-2406731475
5685,2024-11-06,12.52,12.55,12.34,12.50,11.847027,39053800,ABEV3.SA,-0.180000,12.827026,NaN,12.50,1.0,14.22,12.965852,0.180000,0.00,29.632457,12.34,13.09,64.936745,-35.063255,0.061432,0.089519,-0.004732,-2445785275
5686,2024-11-07,12.39,12.69,12.39,12.55,11.894415,23946600,ABEV3.SA,0.050000,12.813174,NaN,12.55,1.0,14.21,13.028059,0.000000,0.05,34.532486,12.34,13.09,63.089908,-36.910092,0.053228,0.082261,-0.005583,-2469731875
5687,2024-11-08,12.40,12.55,12.31,12.33,11.685907,35013400,ABEV3.SA,-0.220000,12.789016,NaN,12.33,1.0,14.27,13.090156,0.220000,0.00,25.512916,12.31,13.09,61.412241,-38.587759,0.044266,0.074662,-0.007477,-2504745275
5688,2024-11-11,12.31,12.55,12.26,12.51,11.856504,20684800,ABEV3.SA,0.180000,12.775065,NaN,12.51,1.0,13.80,13.125648,0.000000,0.18,40.246796,12.26,13.09,62.055998,-37.944002,0.035627,0.066855,-0.008924,-2525430075
5689,2024-11-12,12.43,12.64,12.30,12.58,11.922848,26899600,ABEV3.SA,0.070000,12.765312,NaN,12.58,1.0,13.10,13.124366,0.000000,0.07,45.118006,12.26,13.09,60.880907,-39.119093,0.027675,0.059019,-0.009833,-2552329675
5690,2024-11-13,12.52,12.77,12.46,12.73,12.065012,30341700,ABEV3.SA,0.150000,12.763546,NaN,12.73,1.0,13.08,13.122148,0.000000,0.15,54.324617,12.26,13.09,60.668183,-39.331817,0.020988,0.051413,-0.009002,-2582671375
5691,2024-11-14,12.69,12.90,12.61,12.64,11.979713,38027500,ABEV3.SA,-0.089999,12.757369,NaN,12.64,1.0,12.84,13.108040,0.089999,0.00,48.672047,12.26,13.09,59.923931,-40.076069,0.015017,0.044134,-0.008174,-2620698875


In [21]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2000-01-05'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.head(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.260847,75,ABEV3.SA,-0.029034,0.509553,NaN,0.485255,1.0,0.639548,0.545296,0.029034,0.000000,43.244009,0.468107,0.527474,69.813010,-30.186990,-0.000988,-0.000683,-0.010422,15246
1,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.265805,455,ABEV3.SA,0.009223,0.508799,NaN,0.494478,1.0,0.657994,0.550931,0.000000,0.009223,47.299662,0.468107,0.527474,68.543373,-31.456627,-0.001096,-0.000769,-0.013089,14791
2,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.262261,5005,ABEV3.SA,-0.006593,0.507753,NaN,0.487885,1.0,0.652734,0.556021,0.006593,0.000000,44.667003,0.468107,0.527474,66.781944,-33.218056,-0.001227,-0.000863,-0.015941,9786
3,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.255173,3033,ABEV3.SA,-0.013185,0.506101,NaN,0.474700,1.0,0.652734,0.560856,0.013185,0.000000,39.583155,0.468107,0.527474,63.998121,-36.001879,-0.001415,-0.000976,-0.017728,6753
4,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.251629,11602,ABEV3.SA,-0.006593,0.504201,NaN,0.468107,1.0,0.606585,0.563143,0.006593,0.000000,37.143972,0.468107,0.527474,60.798215,-39.201785,-0.001661,-0.001116,-0.016919,-4849
5,2000-03-03,0.512957,0.512957,0.512957,0.512957,0.275738,758,ABEV3.SA,0.044850,0.504639,NaN,0.512957,1.0,0.626363,0.566304,0.000000,0.044850,57.635161,0.468107,0.527474,61.535654,-38.464346,-0.001801,-0.001255,-0.011752,-4091
6,2000-03-09,0.527474,0.527474,0.527474,0.527474,0.283542,1971,ABEV3.SA,0.014517,0.505781,NaN,0.527474,1.0,0.626363,0.569307,0.000000,0.014517,62.233396,0.468107,0.527474,63.458871,-36.541129,-0.001814,-0.001368,-0.010509,-2120
7,2000-03-14,0.494478,0.494478,0.494478,0.494478,0.265805,227,ABEV3.SA,-0.032996,0.505215,NaN,0.494478,1.0,0.619771,0.571830,0.032996,0.000000,48.443694,0.468107,0.527474,62.506941,-37.493059,-0.001839,-0.001463,-0.010642,-2347
8,2000-03-17,0.501104,0.501104,0.501104,0.501104,0.269367,1516,ABEV3.SA,0.006626,0.505010,NaN,0.501104,1.0,0.593400,0.572909,0.000000,0.006626,50.961417,0.468107,0.527474,62.160663,-37.839337,-0.001850,-0.001541,-0.011397,-3863
9,2000-03-22,0.520882,0.520882,0.520882,0.520882,0.279998,7583,ABEV3.SA,0.019778,0.505804,NaN,0.520882,1.0,0.586808,0.573604,0.000000,0.019778,58.021804,0.468107,0.527474,63.497441,-36.502559,-0.001783,-0.001590,-0.007358,3720


In [22]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_Ambev.pkl')

In [23]:
Features = test_day_AMBEV[['RSI','k_percent','r_percent','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent,r_percent,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
0,43.244009,69.813010,-30.186990,-0.010422,-0.000988,-0.000683,15246
1,47.299662,68.543373,-31.456627,-0.013089,-0.001096,-0.000769,14791
2,44.667003,66.781944,-33.218056,-0.015941,-0.001227,-0.000863,9786
3,39.583155,63.998121,-36.001879,-0.017728,-0.001415,-0.000976,6753
4,37.143972,60.798215,-39.201785,-0.016919,-0.001661,-0.001116,-4849


In [24]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf_Ambev.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

# Visualizar o DataFrame com a nova coluna
test_day_AMBEV.head()

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_20,Smoothed_Close_20,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
0,2000-02-22,0.485255,0.485255,0.485255,0.485255,0.260847,75,ABEV3.SA,-0.029034,0.509553,1.0,0.485255,1.0,0.639548,0.545296,0.029034,0.000000,43.244009,0.468107,0.527474,69.813010,-30.186990,-0.000988,-0.000683,-0.010422,15246
1,2000-02-23,0.494478,0.494478,0.494478,0.494478,0.265805,455,ABEV3.SA,0.009223,0.508799,1.0,0.494478,1.0,0.657994,0.550931,0.000000,0.009223,47.299662,0.468107,0.527474,68.543373,-31.456627,-0.001096,-0.000769,-0.013089,14791
2,2000-02-24,0.487885,0.487885,0.487885,0.487885,0.262261,5005,ABEV3.SA,-0.006593,0.507753,1.0,0.487885,1.0,0.652734,0.556021,0.006593,0.000000,44.667003,0.468107,0.527474,66.781944,-33.218056,-0.001227,-0.000863,-0.015941,9786
3,2000-02-25,0.474700,0.474700,0.474700,0.474700,0.255173,3033,ABEV3.SA,-0.013185,0.506101,1.0,0.474700,1.0,0.652734,0.560856,0.013185,0.000000,39.583155,0.468107,0.527474,63.998121,-36.001879,-0.001415,-0.000976,-0.017728,6753
4,2000-02-29,0.468107,0.468107,0.468107,0.468107,0.251629,11602,ABEV3.SA,-0.006593,0.504201,1.0,0.468107,1.0,0.606585,0.563143,0.006593,0.000000,37.143972,0.468107,0.527474,60.798215,-39.201785,-0.001661,-0.001116,-0.016919,-4849


In [25]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)